In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content
!git clone https://github.com/ErikEnglesson/SGN.git

In [ ]:
# Installing Conda
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
! conda init
! source ~/.bashrc

In [ ]:
# clear all envs
import subprocess

def list_conda_envs():
    """List all conda environments except the base environment."""
    try:
        result = subprocess.run(['conda', 'env', 'list'], stdout=subprocess.PIPE, text=True, check=True)
        envs_output = result.stdout
        # Parse the output to extract environment names
        env_names = []
        for line in envs_output.splitlines():
            if line.startswith('#') or not line.strip():
                continue
            if 'base' in line and '*' in line:
                # Skip the base environment
                continue
            env_name = line.split()[0]
            env_names.append(env_name)
        return env_names
    except subprocess.CalledProcessError as e:
        print(f"Error listing conda environments: {e.stderr}")
        return []

def delete_conda_env(env_name):
    """Delete a conda environment by name."""
    try:
        print(f"Deleting environment: {env_name}")
        subprocess.run(['conda', 'env', 'remove', '--name', env_name, '-y'], check=True)
        print(f"Environment '{env_name}' deleted successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error deleting environment '{env_name}': {e.stderr}")

def delete_all_conda_envs():
    """Delete all conda environments except the base environment."""
    envs_to_delete = list_conda_envs()
    if not envs_to_delete:
        print("No environments found to delete, except the base environment.")
    else:
        for env in envs_to_delete:
            delete_conda_env(env)
        print("All non-base environments have been deleted.")

# Run the function to delete all environments
delete_all_conda_envs()


In [ ]:
import os
import subprocess

# Environment name and version definitions
env_name = "ub-reprod-exact"
python_version = "3.9.7"
cuda_version = "11.2"
tensorflow_version = "2.6.0"
requirements_file = "/content/SGN/requirements.txt"

# Command definitions
create_env_cmd = f"conda create -n {env_name} python={python_version} -y"
install_cuda_cmd = f"conda run -n {env_name} CONDA_OVERRIDE_CUDA='{cuda_version}' conda install cudatoolkit={cuda_version} -c conda-forge -y"
install_tf_cmd = f"conda run -n {env_name} conda install tensorflow={tensorflow_version} -c conda-forge -y"
pip_install_cmd = f"conda run -n {env_name} pip install -r {requirements_file}"

def run_command(command, description):
    """Run a command with bash and capture errors"""
    print(f"Running: {description}")
    try:
        result = subprocess.run(f"bash -c '{command}'", shell=True, check=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error during: {description}")
        print(f"Exit Code: {e.returncode}")
        print(f"Error Output: {e.stderr}")
        exit(1)

# Check if the environment already exists
def environment_exists(env_name):
    result = subprocess.run(["conda", "env", "list"], stdout=subprocess.PIPE, text=True)
    return env_name in result.stdout

# Create the environment and install libraries in sequence if it doesn't exist
if not environment_exists(env_name):
    run_command(create_env_cmd, f"Creating environment '{env_name}' with Python {python_version}")
    run_command(install_cuda_cmd, f"Installing CUDA Toolkit {cuda_version}")
    run_command(install_tf_cmd, f"Installing TensorFlow {tensorflow_version}")
else:
    print(f"Environment '{env_name}' already exists. Skipping creation and initial installation.")

# Install the requirements file if it exists
if os.path.exists(requirements_file):
    run_command(pip_install_cmd, f"Installing requirements from {requirements_file}")
else:
    print(f"Requirements file {requirements_file} not found. Skipping pip install.")

print("Installation complete.")


In [ ]:
''' # Modified command to run the experiment
experiment_cmd = (
    f"conda run -n ub-reprod-exact python /content/SGN/src/cifar/sgn.py "
    f"--data_dir=/content/SGN/src/cifar/datasets/ "
    f"--output_dir=/content/SGN/src/cifar/output/ "
    f"--dataset cifar100 "
    f"--noisy_labels "
    f"--corruption_type sym "
    f"--severity 0.4 "
    f"--download_data"
)

def run_experiment():
    """Run the experiment command with conda environment."""
    try:
        result = subprocess.run(experiment_cmd, shell=True, check=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error running the experiment:")
        print(f"Exit Code: {e.returncode}")
        print(f"Error Output: {e.stderr}")
        exit(1)

run_experiment() '''

python_path = "/usr/local/envs/ub-reprod-exact/bin/python"
%cd /content/SGN
!{python_path} src/cifar/sgn.py --data_dir=/src/cifar/datasets/ \
                        --output_dir=/src/cifar/output/ \
                        --dataset cifar10 \
                        --download_data \
                        --train_epochs=100
                        #--severity 0.4 \
                        #--noisy_labels \
                        #--corruption_type sym \
                        #--train_proportion=0.09 \